In [ ]:
# MIP

import os
import numpy as np
import cupy as cp
import cv2
import matplotlib.pyplot as plt
from skimage.io import imread, imsave
from cupyx.scipy.ndimage import minimum_filter
import re


def rolling_ball_gpu(image, radius):
    """
    Using CuPy
    """
    kernel_size = int(2 * radius + 1)
    # minimam filter
    gpu_background = minimum_filter(image, size=kernel_size)
    # remove background
    gpu_result = image - gpu_background
    # clip of negative values
    gpu_result = cp.clip(gpu_result, 0, None)
    return gpu_result

# setting image directory
origin_dir = r"your origin directory"
save_dir = r"green_mip"
os.makedirs(save_dir, exist_ok=True)

# get list of image dir
sample_images = [os.path.join(origin_dir, tif) for tif in os.listdir(origin_dir) if tif.endswith(".tif")]

for i, sample_image in enumerate(sample_images):
    # process each sample image directory
    image_file = os.path.basename(sample_image)
    sample_name = image_file.split(".")[0]
    sample_label_name = re.split(r'[-]', sample_name)
    sample_name_filter_list = re.split(r'[-_]',sample_name)


    if "nx" in sample_name_filter_list:

        sample_label_name = "-".join(sample_name_filter_list[6:9])

    elif "during" in sample_name_filter_list:
        sample_label_name = sample_name_filter_list[3]

    save_label_name = "-".join(sample_name_filter_list[2].split('_')[0:3])


    # Image loading and Z-stack integration
    z_stack = imread(sample_image)

    # Apply background processing to each slice
    radius = 50  # Rolling ball radius
    processed_slices = []

    for slice_idx, slice_image in enumerate(z_stack):

        # Transfer the image to the GPU
        gpu_image = cp.asarray(slice_image)

        # Gaussian Filter (GPU)
        gpu_blurred = cv2.GaussianBlur(cp.asnumpy(gpu_image), (0, 0), sigmaX=2)

        # Rolling ball background removal (GPU)
        gpu_result = rolling_ball_gpu(cp.asarray(gpu_blurred), radius=300)

        # save result as cpu format
        result_image = cp.asnumpy(gpu_result)

        processed_slices.append(result_image)

    # Convert background processed Z-stack to NumPy array
    processed_z_stack = np.array(processed_slices)

    # MIP
    mip_image = np.max(processed_z_stack, axis=0)

    # **Check the shape of the mip_image and make it 2D**
    if mip_image.ndim == 3:
        mip_image = mip_image[:, :, 1]  # Get only the green channel
    elif mip_image.ndim != 2:
        raise ValueError(f"Unexpected MIP image shape: {mip_image.shape}")

    # **to RGB image**
    mip_green = (mip_image / mip_image.max() * 255).astype(np.uint8)  # Normalized to 0-255
    mip_rgb = cv2.merge([np.zeros_like(mip_green), mip_green, np.zeros_like(mip_green)])  # (H, W, 3)

    # **save**
    output_path = os.path.join(save_dir, f"{sample_label_name}_mip-green.tif")
    imsave(output_path, mip_rgb)

    print(f"Processed {sample_name} and saved to {output_path}")


In [ ]:
# FFT

import cv2
import numpy as np
import matplotlib.pyplot as plt
import re

def gaussian_high_pass_filter(image, cutoff=1000):
    """
    Apply Gaussian High-Pass Filter (GHPF) to a 2D image
    Args: 
        image (numpy.ndarray): Grayscale image
        cutoff (int): Cutoff frequency (larger value leaves more high frequencies)
    Returns:
        numpy.ndarray: Image after high frequency emphasis
    """
    # Fourier transform of an image
    dft = np.fft.fft2(image)
    dft_shift = np.fft.fftshift(dft)  # Centering on frequency components

    # Get Image Size
    rows, cols = image.shape
    crow, ccol = rows // 2, cols // 2  # image center coordinates

    # Creating a Gaussian High-Pass Filter
    x, y = np.meshgrid(np.linspace(-1, 1, cols), np.linspace(-1, 1, rows))
    radius = np.sqrt(x**2 + y**2)
    ghpf = 1 - np.exp(- (radius * cutoff) ** 2)  # emphasize high frequencies

    # Apply filter
    dft_shift_filtered = dft_shift * ghpf

    # Return to the image using the inverse Fourier transform
    dft_inverse_shift = np.fft.ifftshift(dft_shift_filtered)
    image_filtered = np.fft.ifft2(dft_inverse_shift)
    image_filtered = np.abs(image_filtered)

    # Normalize and scale to 0-255
    image_filtered = cv2.normalize(image_filtered, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    return image_filtered

# Specifying the image directory
origin_dir = r"green_mip"
save_dir = r"fft"
os.makedirs(save_dir, exist_ok=True)

# Get list of TIFF images
sample_images = [os.path.join(origin_dir, tif) for tif in os.listdir(origin_dir) if tif.endswith(".tif")]

for i, sample_image in enumerate(sample_images):
    # sample image director
    image_file = os.path.basename(sample_image)
    sample_name = image_file.split(".")[0]
    sample_label_name = re.split(r'[-]', sample_name)
    sample_name_filter_list = re.split(r'[-_]',sample_name)

    # Image loading and Z-stack integration
    image = imread(sample_image)

    g_channnel = image[:,:,1]

    fft_image = gaussian_high_pass_filter(g_channnel)

    # save
    output_path = os.path.join(save_dir, f"{sample_name}_fft_filter.tif")
    imsave(output_path, fft_image)

    print(f"Processed {sample_name} and saved to {output_path}")

Plese make ROI of cells using "make_extract_ROI.py"

In [ ]:
# extract cell

import tifffile as tiff
import json
import os
import numpy as np
from skimage.draw import polygon
from skimage.color import rgb2gray
from skimage.measure import label
import re

from tqdm import tqdm


def load_shapes_json(json_path):
    """Load ROI data from JSON"""
    with open(json_path, "r") as f:
        data = json.load(f)
    return data["rois"]


def extract_roi_regions(image, roi_shapes):
    """
    Create a new image for each ROI (fill in black outside the ROI)
    :param image: 2D image (H, W)
    :param roi_shapes: A list of [[(x1, y1), (x2, y2), ...], ...]
    :return: Mask image list for each ROI
    """
    masked_images = []  # A list to save masked images
    extracted_images = []  # A list to store images for each ROI

    for roi in roi_shapes: 
        masked_image = np.zeros_like(image) 
        
        roi_np = np.array(roi)
        rr, cc = polygon(roi_np[:, 0], roi_np[:, 1], shape=image.shape) 
        
        # Apply mask (ROI area to original pixel values)
        masked_image[rr, cc] = image[rr, cc]
        
        # Get the minimum and maximum coordinates of the ROI region
        min_r, max_r = np.min(rr), np.max(rr)
        min_c, max_c = np.min(cc), np.max(cc)
        
        # Extract ROI area (not including background)
        roi_image = masked_image[min_r:max_r+1, min_c:max_c+1]
        extracted_images.append(roi_image) 

    extracted_images = np.array(extracted_images)
    return extracted_images

# Process
slice_start_index = []

# save directory
result_savedir = r"ROI_extracted_regions"
os.makedirs(result_savedir, exist_ok=True)

# Image directory
image_dir = r"fft"
image_files = [os.path.join(image_dir, tif) for tif in os.listdir(image_dir) if tif.endswith(".tif")]

# Get roi directory
roi_dirs = {}

# Matching images and ROIs for processing
for image_path in image_files:
    
    image_file = os.path.basename(image_path)
    sample_name = image_file.split("_")[0]

    # load image
    image_file = os.path.basename(image_path)
    image = tiff.imread(image_path)

    if len(image.shape) == 3:
        g_channnel = image[:,:,1]
    else:
        g_channnel  = image

    print(f"Processing: {sample_name}")

    roi_sample_dir = "ROI_json" + "/" + sample_name
    roi_files = [os.path.join(roi_sample_dir, roi) for roi in os.listdir(roi_sample_dir) if roi.endswith(".json")]


    for i, json_path in enumerate(tqdm(roi_files)):

        output_path = os.path.join(result_savedir, f"{sample_name}-roi-{i}.tif")
        if os.path.exists(output_path):
            continue

        # Load JSON
        rois = load_shapes_json(json_path)
        # Apply ROI and acquire images for each ROI
        extract_images = extract_roi_regions(g_channnel, rois)

        tiff.imwrite(output_path, extract_images)
        
    
"""
# Saving the acquired image
import tifffile as tiff
import os
import napari


extracted_image_dir = r"ROI_extracted_regions"
extracted_images = [os.path.join(extracted_image_dir, roi) for roi in os.listdir(extracted_image_dir) if roi.endswith(".tif")]

for i, image_path in enumerate(extracted_images):
    image = tiff.imread(image_path)
    
    if i % 10 == 1000:
        print(image_path)
        vi = napari.Viewer()
        vi.add_image(image)
        napari.run()
"""

In [ ]:
import cv2
import numpy as np
import os
from skimage.io import imread, imsave
from skimage.filters import threshold_otsu
from skimage.measure import label, regionprops

# save directory
result_savedir = r"Threshold"
os.makedirs(result_savedir, exist_ok=True)

# image directory
image_dir = r"ROI_extracted_regions"
image_files = [os.path.join(image_dir, tif) for tif in os.listdir(image_dir) if tif.endswith(".tif")]

# Margin pixels
margin = 10

# Matching images and ROIs for processing
for image_path in image_files:
    image_file = os.path.basename(image_path)
    sample_name = image_file.split(".")[0]
    image = imread(image_path)
    image = np.squeeze(image)
    pad_width = ((margin, margin), (margin, margin))

    padded_image = np.pad(image, pad_width=pad_width, mode='constant', constant_values=0)

    # Otsu's thresholding (uses the G channel if grayscale conversion is required)
    if padded_image.ndim == 3:  
        padded_gray = padded_image[:, :, 1]
    else:
        padded_gray = padded_image 

    threshold_value = threshold_otsu(padded_gray)
    binary_image = padded_gray > 40

    # Labeling process
    labeled_image = label(binary_image)
    regions = regionprops(labeled_image)

    # Find the largest area
    if len(regions) > 0:
        largest_region = max(regions, key=lambda r: r.area)
        largest_label = largest_region.label

        # Keep only the largest regions
        largest_component = (labeled_image == largest_label).astype(np.uint8) * 255

        # Remove margins and restore to original size
        if largest_component.ndim == 3:
            final_image = largest_component[margin:-margin, margin:-margin, :]
        else:
            final_image = largest_component[margin:-margin, margin:-margin]

        # save image
        save_path = os.path.join(result_savedir, f"{sample_name}_threshold.tif")
        imsave(save_path, final_image)


Please make ROI of cell body using "make_cell_body_ROI.py"

In [ ]:
# get params

import os
import numpy as np
import cv2
import tifffile as tiff
import pandas as pd
import json

from skimage.measure import perimeter, regionprops, label
from skimage.morphology import convex_hull_image
from scipy.spatial import ConvexHull, distance
from skimage.filters import threshold_otsu
from skimage.morphology import skeletonize
from scipy.stats import linregress
from scipy.ndimage import distance_transform_edt
from skimage.morphology import disk
from skimage.graph import route_through_array
from scipy.spatial.distance import euclidean

from tqdm import tqdm

def box_counting_fractal(binary_img):
    """
    Calculates the fractal dimension of a 2D image using the box-counting method
    :param binary_img: 2D binary image (np.ndarray, dtype=bool)
    :return: Fractal dimension D
    """
    # List of box sizes (2^0, 2^1, ..., 2^9)
    sizes = 2 ** np.arange(10)
    counts = []

    for size in sizes:
        # Calculate the total value for each grid
        grid = np.add.reduceat(
            np.add.reduceat(binary_img, np.arange(0, binary_img.shape[0], size), axis=0),
            np.arange(0, binary_img.shape[1], size), axis=1
        )
        # Count if the box contains at least one pixel
        counts.append(np.count_nonzero(grid))

    # Take the logarithm of box size and counts
    log_sizes = np.log(sizes)
    log_counts = np.log(counts)

    # Calculate the fractal dimension by linear regression (absolute value of the gradient)
    coeffs = np.polyfit(log_sizes, log_counts, 1)
    D = -coeffs[0]

    return D


def lacunarity(binary_img):
    """
    Calculates the lacunarity of a 2D image using the box-counting method
    :param binary_img: 2D binary image (np.ndarray, dtype=bool)
    :return: Lacunarity value
    """
    # List of box sizes (2^0, 2^1, ..., 2^9)
    sizes = 2 ** np.arange(10)
    lacunarity_vals = []

    for size in sizes:
        grid = np.add.reduceat(
            np.add.reduceat(binary_img, np.arange(0, binary_img.shape[0], size), axis=0),
            np.arange(0, binary_img.shape[1], size), axis=1
        )

        # Calculate the mean and variance of pixel counts
        mean_val = np.mean(grid)
        var_val = np.var(grid)

        # Calculate lacunarity (variance/mean^2)
        if mean_val > 0:
            lacunarity_vals.append(var_val / mean_val**2 + 1)

    # Returns the average lacunarity
    return np.mean(lacunarity_vals) if lacunarity_vals else np.nan


def sholl_analysis(skeleton, centroid, max_radius=50, step_size=5):
    """A function that performs a Sholl analysis (counts the number of intersections on a circle)"""
    radii = np.arange(step_size, max_radius + step_size, step_size, dtype=np.float64)
    intersections = []
    skeleton_coords = np.column_stack(np.where(skeleton == 1))
    
    for r in radii:
        mask = np.isclose(np.sqrt(np.sum((skeleton_coords - centroid) ** 2, axis=1)), r, atol=0.5)
        intersections.append(np.sum(mask))
    
    intersections_np = np.array(intersections, dtype=np.float64)
    slope, intercept, r_value, _, _ = linregress(radii, intersections_np)
    
    return radii, intersections, slope, intercept


def get_max_process_length(skeleton):
    """ Find the longest path between the endpoints of a skeleton image """
    skel_coords = np.column_stack(np.where(skeleton > 0))
    if len(skel_coords) == 0:
        return 0

    # Cost map (only pixels on skeleton are passable)
    cost_map = np.full(skeleton.shape, np.inf)
    cost_map[skeleton > 0] = 1  # Set the cost of pixels on the skeleton to 1

    # Get the corner points (points with only one adjacent pixel)
    endpoints = [tuple(p) for p in skel_coords if np.sum(skeleton[max(0, p[0]-1):p[0]+2, max(0, p[1]-1):p[1]+2]) == 2]

    max_length = 0
    longest_path = None

    # Examine all endpoint pairs
    for i in range(len(endpoints)):
        for j in range(i + 1, len(endpoints)):
            path, cost = route_through_array(cost_map, endpoints[i], endpoints[j], fully_connected=True)
            if cost > max_length:
                max_length = cost
                longest_path = path

    return max_length, longest_path

def get_straightness_index(skeleton):
    """ Function to calculate straightness from skeleton image """
    skel_coords = np.column_stack(np.where(skeleton > 0))  # Get skeleton coordinates
    if len(skel_coords) == 0:
        return 0  # Returns 0 if the skeleton does not exist.

    # Cost map (only pixels on skeleton are passable)
    cost_map = np.full(skeleton.shape, np.inf)
    cost_map[skeleton > 0] = 1  # 

    # Get the corner points (points with only one adjacent pixel)
    endpoints = [tuple(p) for p in skel_coords if np.sum(skeleton[max(0, p[0]-1):p[0]+2, max(0, p[1]-1):p[1]+2]) == 2]

    if len(endpoints) < 2:
        return 0

    max_straightness = 0

    # Examine all endpoint pairs
    for i in range(len(endpoints)):
        for j in range(i + 1, len(endpoints)):
            path, cost = route_through_array(cost_map, endpoints[i], endpoints[j], fully_connected=True)
            euclidean_distance = euclidean(endpoints[i], endpoints[j]) 
            if cost > 0:
                straightness = euclidean_distance / cost
                max_straightness = max(max_straightness, straightness)

    return max_straightness


def compute_morphological_features(image_path, pixel_size=0.28, perimeter_pixel=0.3):
    """ Morphological analysis of microglia"""

    # Image loading
    binary_img = tiff.imread(image_path)
    binary_img = binary_img > threshold_otsu(binary_img)
    labeled_img = label(binary_img)

    # Keep only the largest regions
    regions = regionprops(labeled_img)
    if len(regions) == 0:
        print(f"Warning!: No parsable object at {image_path}")
        return {}, None
    
    region = max(regions, key=lambda r: r.area)
    largest_label = region.label
    largest_component = (labeled_img == largest_label).astype(np.uint8) * 255

    Fractal_d = box_counting_fractal(largest_component)
    lacunality_value = lacunarity(largest_component)

    ## Calculate the convex hull
    convex_hull = convex_hull_image(largest_component)
    convex_labels = label(convex_hull.astype(int))
    convex_regions = regionprops(convex_labels)

    if len(convex_regions) == 0:
        print(f"Warning!: No parsable object at {image_path}")
        return {}, None
    
    convex_region = convex_regions[0]


    cell_area = region.area * pixel_size ** 2  # µm²
    convex_area = convex_region.area * pixel_size ** 2
    cell_perimeter = perimeter(largest_component > 0) * perimeter_pixel
    convex_perimeter = perimeter(convex_hull>0) * perimeter_pixel
    density = cell_area / convex_area
    roughness = cell_perimeter / convex_perimeter
    circularity = (4 * np.pi * cell_area) / (cell_perimeter ** 2)
    convex_circularity = (4 * np.pi * convex_area) / (convex_perimeter ** 2)

    """skeleton analyze"""
    skeleton = skeletonize(binary_img)
    labeled_img = label(skeleton)
    regions = regionprops(labeled_img)

    if len(regions) == 0:
        print(f"Warning!: No parsable object at {image_path}")
        return {}

    region = max(regions, key=lambda r: r.area)
    largest_label = region.label
    largest_component = (labeled_img == largest_label).astype(np.uint8) * 255

    # get analysis results
    total_process_length = np.sum(skeleton)
    num_branch_points = np.sum(perimeter(skeleton))
    mean_process_length = total_process_length / max(1, num_branch_points)
    centroid = region.centroid

    # Sholl analysis
    sholl_radii, sholl_intersections, sholl_slope, sholl_intercept = sholl_analysis(skeleton, centroid)
    sholl_max_counts = max(sholl_intersections) 

    # process length
    max_process_length = get_max_process_length(skeleton)

    # straghtness
    straightness = get_straightness_index(skeleton)

    # Calculate 2D distance map
    distance_map = distance_transform_edt(binary_img)

    # Finding the center of the cell body
    max_dist_idx = np.argmax(distance_map)
    cell_center = np.unravel_index(max_dist_idx, distance_map.shape)

    # Obtaining the coordinates of the cell membrane
    cell_boundary = labeled_img == region.label
    boundary_coords = np.array(np.where(cell_boundary)).T

    # Calculate the distance from the center of the cell body to the cell membrane
    distances = np.sqrt(np.sum((boundary_coords - np.array(cell_center))**2, axis=1))

    properties = {
        "Fractal_dimension": Fractal_d,
        "lacunality":lacunality_value,
        "Cell Area (µm²)": cell_area,
        "Convex Hull Area (µm²)": convex_area,
        "Density": density,
        "Cell Perimeter (µm)": cell_perimeter,
        "Convex Hull Perimeter (µm)": convex_perimeter,
        "Roughness": roughness,
        "Convex Hull Circularity": convex_circularity,
        "Cell Circularity": circularity,

        "Total_Process_Length": total_process_length,
        "Mean_Process_Length": mean_process_length,
        "Max_Process_Length": max_process_length[0],
        "Sholl_Max_Counts": sholl_max_counts,
        "Straightness": straightness,
    }
    
    return properties, largest_component


# Analysis directory settings
image_dir = "Threshold"
image_files = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(".tif")]

results_morpho = []

for image_path in tqdm(image_files):
    parameters, largest_component = compute_morphological_features(image_path)
    sample_name = os.path.basename(image_path)
    
    if parameters:
        parameters["sample"] = sample_name.split("_")[0]
        parameters["mouse"] = sample_name.split("_")[0].split("-")[1]

        results_morpho.append(parameters)

def load_shapes_json(json_path):
    """Load ROI data from JSON"""
    with open(json_path, "r") as f:
        data = json.load(f)
    return data["cell_body_rois"]

def calculate_polygon_area(polygon):
    """
    Takes the coordinates of the vertices of a polygon and calculates its area using Shoelace's formula
    polygon: List of [(x1, y1), (x2, y2), ..., (xn, yn)]
    """
    polygon = np.array(polygon)  # Numpy
    x = polygon[:, 0]
    y = polygon[:, 1]

    # Shoelace Official
    area = 0.5 * np.abs(np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1)))
    return area

# Analysis directory settings for cell body
ROI_dir = "ROI_json_cell_body"
roi_files = [os.path.join(ROI_dir, f) for f in os.listdir(ROI_dir) if f.endswith(".json")]

results_cell_body_area = []
sample_list = []

for roi_path in tqdm(roi_files):
    sample_name = os.path.basename(roi_path).split('.')[0].split("_")[0]
    cell_body_rois = load_shapes_json(roi_path)

    # Sum the area of ​​all ROIs
    total_area = sum(calculate_polygon_area(roi) for roi in cell_body_rois)

    # Convert pixels to μm²
    pixel_size = 0.28  # please enter your pixel size
    area_um2 = total_area * (pixel_size ** 2)

    results_cell_body_area.append(area_um2)
    sample_list.append(sample_name) 

# Creating a data frame
df_cell_body = pd.DataFrame({"sample": sample_list, "Cell body Area (μm²)": results_cell_body_area})

# save
if results_morpho:
    df = pd.DataFrame(results_morpho)
    df = pd.merge(df, df_cell_body, on="sample")
    csv_path ="morphology_results_Iba1.csv"
    df.to_csv(csv_path, index=False)
    print(f"The morphology analysis results have been saved in {csv_path}")
